In [ ]:

import numpy as np
import seaborn as sns
import pandas as pd
import os
import matplotlib.pyplot as plt
from matplotlib.image import imread
import tensorflow

# Data_explore and read in data 

'''Input your data path in Data Path'''

data_dir = r"Data path"
(os.listdir(data_dir))

test_path = data_dir + '\\test'
train_path = data_dir + '\\train'

os.listdir(test_path)
os.listdir(train_path)

var = (os.listdir(train_path + '\\parasitized')[0])
var2 = (os.listdir(train_path + '\\uninfected')[0])

para_cell = train_path + '\\parasitized\\' + var
uninfected_cell = train_path + '\\uninfected\\' + var2

imread(para_cell)
# print(imread(para_cell).shape)
# plt.show(para_cell)

imread(uninfected_cell)
# print(imread(uninfected_cell).shape)

infected_number = len(os.listdir(train_path + '\\parasitized'))
uninfected_number = len(os.listdir(train_path + '\\uninfected'))

dim1 = []
dim2 = []

for image_filename in os.listdir(test_path + '\\uninfected'):
    img = imread(test_path + '\\uninfected\\' + image_filename)
    d1, d2, colors = img.shape
    dim1.append(d1)
    dim2.append(d2)

print(np.mean(dim1), np.mean(dim2))

image_shape = (130, 130, 3)



In [ ]:

# ImageDataGenerator:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

image_gen = ImageDataGenerator(rotation_range=20,
                               width_shift_range=0.1,
                               height_shift_range=0.1,
                               rescale=1,
                               shear_range=0.1,
                               zoom_range=0.1,
                               horizontal_flip=True,
                               fill_mode='nearest')



In [ ]:
#TEST
image_gen.flow_from_directory(train_path)
image_gen.flow_from_directory(test_path)

# Model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import MaxPool2D, Dense, Dropout, Conv2D, Flatten, Lambda
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import SGD, Nadam



model = Sequential()

model.add(Conv2D(kernal_size =(3, 3), filters=32, input_shape=image_shape, activation='relu'))
model.add(MaxPool2D(pool_size=(2, 2)))

model.add(Conv2D(kernal_size =(3, 3), filters=64, input_shape=image_shape, activation='relu'))
model.add(MaxPool2D(pool_size=(2, 2)))

model.add(Lambda(lambda v: tensorflow.cast(tensorflow.spectral.fft(tensorflow.cast(v, dtype = tensorflow.complex64)), tensorflow.float32)))

model.add(Conv2D(kernal_size =(3, 3), filters=64, input_shape=image_shape, activation='relu'))
model.add(MaxPool2D(pool_size=(2, 2)))

model.add(Flatten())

model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))

model.add(Dense(1), activation='sigmoid')

model.compile(loss= 'binary_crossentrophy', optimizer=Nadam,
              metrics=['accuracy'])

model.Summary()


early_stop = EarlyStopping(moniter='val_loss', patience=2)
batch_size = 64


train_image_gen = image_gen.flow_from_directory(train_path,
                                                target_size=image_shape[:2],
                                                color_mode='rgb',
                                                batch_size=batch_size,
                                                class_mode='binary')

test_image_gen = image_gen.flow_from_directory(test_path,
                                               target_size=image_shape[:2],
                                               color_mode='rgb',
                                               batch_size=batch_size,
                                               class_mode='binary',
                                               shuffle=False)

model_results = model.fit_generator(train_image_gen, epochs=20,
                                    validation_data=test_image_gen,
                                    callbacks=[early_stop])


model.save("MalariaDetector.h5")

#model.save is a tensorflow.keras object created using the above methods

In [ ]:

# PretrainedModel(Model):

from tensorflow.keras.models import load_model
from sklearn.metrics import classification_report, confusion_matrix
from tensorflow.keras.preprocessing import image

model = load_model('malaria_detector.h5')
model.summary()
model.evaluate_generator(test_image_gen)




In [ ]:
# Predict classification of random image

pred = model.predict_generator(test_image_gen)

# Define the confidence level of classification
predictions = pred > 0.8

print(classification_report(test_image_gen.classes, predictions))
confusion_matrix(test_image_gen.classes, predictions)

my_image = image.load_img(para_cell, target_size=image_shape)

